<a href="https://colab.research.google.com/github/adong-hood/cs200/blob/main/assignment5_EDA_ch63.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Use Web API

Using a Web API allows programs to access data or services over the internet. In Python, the requests library is commonly used to send HTTP requests to APIs and retrieve information in formats like JSON.

### This section covers a few large concepts that work together in one big example:
<ul>
    <li>Using a web API to get information</li>
    <li>Applying a function to a data set and getting a new column</li>
    <li>Mapping data using Altair</li>
</ul>

Background Reading: <a href = "https://nextjournal.com/sdanisch/cartographic-visualization">Cartographic Visualization</a>

## Imports

In [ ]:
import altair as alt
import pandas as pd
from vega_datasets import data #- # Imports the 'data' module from vega_datasets to access built-in example datasets for visualization
import requests  #- for web API
# for Jupyter Notebook only. alt.renderers.enable('notebook')

Note: Vega Datasets is a common repository for example datasets used by Vega related projects. Vega is a visualization grammar, a declarative language for creating, saving, and sharing interactive visualization designs. With Vega, you can describe the visual appearance and interactive behavior of a visualization in a JSON format, and generate web-based views using Canvas or SVG.

## Warmup Example
Let's take on the seemingly simple task of plotting some of the country data on a map. <br>
Altair provides us with the facility to make a blank map. The counties data that is passed to the chart is the data needed to create and outline the map.

**altair.topo_feature  **
<code>altair.topo_feature(url, feature, **kwargs)</code>
A convenience function for extracting features from a **topojson url**. A TopoJSON URL is simply a web link (URL) that points to a TopoJSON file — a geographic data format used mainly for maps.  
**Parameters:**

- **url**:string  An URL from which to load the data set.
- **feature**:string The name of the TopoJSON object set to convert to a GeoJSON feature collection. For example, in a map of the States, there may be an object set named “counties”. Using the feature property, we can extract this set and generate a GeoJSON feature object for each county.
- **\*\*kwargs**: additional keywords passed to TopoDataFormat


In [ ]:
counties = alt.topo_feature(data.us_10m.url, 'counties')  # Load TopoJSON data and extract county features

alt.Chart(counties).mark_geoshape().project(               # Create chart with geographic shapes
    type='albersUsa'                                       # Use Albers USA map projection to fit all US states in one map.
).properties(
    width=500,                                             # Set chart width (pixels)
    height=300                                             # Set chart height (pixels)
)


What we want to do is to graph/overlay the unemployment data by county.

In [ ]:
unemp_data = data.unemployment.url
unemp_data

In [ ]:
unemp_data = pd.read_csv(unemp_data,sep='\t')
unemp_data.head()

Using the <code>transform_lookup</code> method, we can arrange for the id in the geographic data to be matched against the id in our unemp_data data frame.

In [ ]:
alt.Chart(counties).mark_geoshape(           #- Create blank map with geographic shapes
).encode(
    color='rate:Q'                           #- Color counties by unemployment rate (quantitative)
).transform_lookup(
    lookup='id',                             #- Match county id field from county map
    from_=alt.LookupData(unemp_data, 'id', ['rate'])  # extract unemployment rate by id from unemp_data
).project(
    type='albersUsa'                         #- Use U.S. map projection
).properties(
    width=500,                               #- Chart width in pixels
    height=300,                              #- Chart height in pixels
    title='Unemployment by County'           #- Chart title
)


## A More Complete Example


###Part 1 - Adding country codes to our data.  
Let's try a more complicated case where we do not have all the data readily available to us. This time, we will plot the average incomes of countries on a map.  
**Read in country income data**: This dataset contains the average income for a few countries, but it does not include each country's numeric code, which is needed for transform_lookup when mapping income on a map.

Our goal is to add a unique numeric country code to this dataset so that we can map the data.

In [ ]:
income_data = pd.read_csv('http://pluto.hood.edu/~dong/datasets/country_income.csv').dropna(axis = 1, how = "all")
print(income_data.shape)
income_data.head(2)

**Look up country code for one country**: We need to find countries' numeric code on line. We import the requests module, then equest what we want, and save the results as res.

In [ ]:
#requst
res = requests.get('https://restcountries.com/v2/alpha/aus')  # Send API request for Australia data
res.status_code  # 200 means the request succeeded

200

<code>get </code> method does the following - first it goes to the website https://restcountries.com/v2/alpha/usa, then returns the information for that country in json format.  

<!--li>/rest - technically REST stands for REpresentational State Transfer. This uses the HTTP protocol to ask for and respond with data.</li-->
- **v2** - this is version 2 of this website’s protocol
- **alpha** - This tells the website that the next thing we are going to pass tell it is the three letter code for the country.
- **AUS** - this can be any valid three letter country code. for example, USA.

In [ ]:
res.text              #- Raw response text returned by the API (JSON as a string)
temp = res.json()     #- Convert JSON response into a Python dictionary
temp                  #- Display the parsed country data dictionary
temp['numericCode']   #- Extract the country’s numeric code (e.g., Australia = 036)


**Adding country code to income data:** We now implement a function that takes a country letter code and fetch its numeric code from the web using Web API. This will allow us to add numeric code to all countries easily.

In [ ]:
#function, end point v2.
def look_up_code(country_code):
    address = 'https://restcountries.com/v2/alpha/'+country_code
    res = requests.get(address)
    country_info = res.json()
    country_num = country_info['numericCode']
    return int(country_num)

<p><code>map</code> is a method of a Series, so we use the syntax df.myColumn.map(function). This applies the function we pass as a parameter to each element of the series and constructs a brand new series. Add a new column <code>country code</code> with the new series after map.</p>

In [ ]:
#map and add another column
income_data['countrycode'] = income_data.LOCATION.map(look_up_code)  #- Map country codes using lookup function
print(income_data.shape)
income_data.head()

In [ ]:
# country with no income data
income_data[income_data['LOCATION'] == 'BRA']

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,countrycode


### Part 2 - Mapping Income Data

In [ ]:
#- 2.1. Getting a Blank Map
#- get the countries objects.
countries = alt.topo_feature(data.world_110m.url, 'countries')
print(type(countries))

The above code indicates that we want to extract Geo features from the specified url for the countries object. All countries are represented using numeric code. Let us draw the world map by passing countries to Altair.

Projections map from a data domain (spatial position) to a visual range (pixel position). We can also specify projection parameters, such as scale (zoom level) and translate (panning), to customize the projection settings.

In [ ]:
#blank map
alt.Chart(countries).mark_geoshape(
    fill='#666666',
    stroke='white'
).properties(
    width=750,
    height=450
).project('equirectangular')

2.2. Adding Income Data to the Map: To encode income data in the geo shape, using the <code>transform_lookup</code> method, we can arrange for country in the geographic data to be matched against the country in our income data frame.

In [ ]:
alt.Chart(countries).mark_geoshape(stroke='black', strokeWidth=0.5).encode(   #- Create world map with country borders
    tooltip='LOCATION:N',                                                     #- Show country name on hover
    color=alt.Color('Value:Q', scale=alt.Scale(scheme='plasma'))              #- Color countries by income value
).transform_lookup(
    lookup='id',                                                              #- Match map country id
    from_=alt.LookupData(income_data, 'countrycode', ['Value', 'LOCATION'])   #- Extract Value and LOCATION data from income_data by countrycode
).project(
    type='equirectangular'                                                    #- Use simple world map projection
).properties(
    width=750,
    height=450,
    title="Income by Country"
)


The big white space indicates we only have income data for a small number of countries.

In [ ]:
#use a different end point.
code = 'USA'
url = f'https://restcountries.com/v3.1/alpha?codes={code}'
res = requests.get(url)
res.status_code

200

In [ ]:
print(res.json()[0]['ccn3'])
res.json()

In [ ]:
#function end point 3. This is new link.
def look_up_code_v2(code):
    address = f'https://restcountries.com/v3.1/alpha?codes={code}'
    res = requests.get(address)
    # Check if the request was successful (status code 200)
    if res.status_code == 200:
        country_info = res.json()[0]
        country_num = country_info['ccn3']
        return int(country_num)
    else:
        # Handle cases where the API request fails
        print(f"Error looking up code for {code}: Status code {res.status_code}")
        # Returning a default value or raising an exception is recommended
        return None  # or raise ValueError(f"Invalid country code: {country_code}")

## Practice

You may refer to Section 6.3 for more details and solutions to some of the questions.

In [ ]:
wd = pd.read_csv('http://pluto.hood.edu/~dong/datasets/world_countries.csv')
print(wd.shape)
print(wd.columns)
wd.head(2)

### Q-1: What is the numericCode for the country of Peru?

### Q-2: Find the list of the three letter country codes of the countries that border Peru. Do not include the square brackets.

### Q-3: How many keys are in the dictionary returned for the country of Peru?

### Q-4: Graph Infant Mortality rates of countries on a Map.